In [31]:
import math
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

import sys
print sys.path
%matplotlib inline

['', '/Users/rtindru/anaconda/lib/python2.7/site-packages/seaborn-0.7.0-py2.7.egg', '/Users/rtindru/anaconda/lib/python27.zip', '/Users/rtindru/anaconda/lib/python2.7', '/Users/rtindru/anaconda/lib/python2.7/plat-darwin', '/Users/rtindru/anaconda/lib/python2.7/plat-mac', '/Users/rtindru/anaconda/lib/python2.7/plat-mac/lib-scriptpackages', '/Users/rtindru/anaconda/lib/python2.7/lib-tk', '/Users/rtindru/anaconda/lib/python2.7/lib-old', '/Users/rtindru/anaconda/lib/python2.7/lib-dynload', '/Users/rtindru/anaconda/lib/python2.7/site-packages/Sphinx-1.3.5-py2.7.egg', '/Users/rtindru/anaconda/lib/python2.7/site-packages/setuptools-20.3-py2.7.egg', '/Users/rtindru/anaconda/lib/python2.7/site-packages', '/Users/rtindru/anaconda/lib/python2.7/site-packages/aeosa', '/Users/rtindru/anaconda/lib/python2.7/site-packages/IPython/extensions', '/Users/rtindru/.ipython']


In [43]:
df_train = pd.read_csv('../Kaggle_Datasets/Facebook/train.csv')
df_test = pd.read_csv('../Kaggle_Datasets/Facebook/test.csv')

In [33]:
mini_df = df_train[(df_train.x > 7.0) & (df_train.x < 7.5) & (df_train.y > 3.0) & (df_train.y < 3.25)]

In [34]:
df_train = None
df_test = None

In [35]:
def mean_diff(df):
    unique_places = df.place_id.unique()
    means = {}
    
    for place_id in unique_places:
        df_place = df[df.place_id == place_id]
        xmean = df_place.x.mean()
        ymean = df_place.y.mean()
        means[place_id] = (xmean, ymean)

    def apply_mean_x(place_id):
        return means[place_id][0]
    
    def apply_mean_y(place_id):
        return means[place_id][1]
    
    df['xmean'] = df.place_id.apply(apply_mean_x)
    df['ymean'] = df.place_id.apply(apply_mean_y)
    df['xdist'] = np.abs(df.x - df.xmean)
    df['ydist'] = np.abs(df.y - df.ymean)
    
    return df

In [37]:
def mod_df(mini_df):
    mini_df.loc[:, 'hour'] = mini_df.time // 60 % 24 + 1
    mini_df.loc[:, 'day'] = mini_df.time // (60*24) % 7 + 1
    mini_df.loc[:, 'week'] = mini_df.time // (60*24*7) % 52 + 1
    mini_df.loc[:, 'month'] = mini_df.time // (60*24*30) % 12 + 1
    mini_df.loc[:, 'year'] = mini_df.time // (60*24*365) + 1

    mini_df.loc[:, 'hours'] = mini_df.time // 60 + 1
    mini_df.loc[:, 'days'] = mini_df.time // (60*24) + 1
    mini_df.loc[:, 'weeks'] = mini_df.time // (60*24*7) + 1
    mini_df.loc[:, 'months'] = mini_df.time // (60*24*30) + 1
    mini_df.loc[:, 'years'] = mini_df.time // (60*24*365) + 1
    
    mini_df.loc[:, 'hour_float'] = mini_df.time / 60 % 24 + 1
    mini_df.loc[:, 'day_float'] = mini_df.time / (60*24) % 7 + 1
    mini_df.loc[:, 'week_float'] = mini_df.time / (60*24*7) % 52 + 1
    mini_df.loc[:, 'month_float'] = mini_df.time / (60*24*30) % 12 + 1
    mini_df.loc[:, 'year_float'] = mini_df.time / (60*24*365) + 1

    mini_df.loc[:, 'log2_accuracy'] = np.log2(mini_df.accuracy) * 10
    mini_df.loc[:, 'log10_accuracy'] = np.log10(mini_df.accuracy) * 10
    mini_df.loc[:, 'log2_accuracy_int'] = (np.log2(mini_df.accuracy) * 10).astype(int)
    mini_df.loc[:, 'log10_accuracy_int'] = (np.log10(mini_df.accuracy) * 10).astype(int)
    
    return mini_df

In [38]:
mini_df = mod_df(mini_df)
#mini_df = mean_diff(mini_df)
mini_df.head()

,row_id,x,y,accuracy,time,place_id,hour,day,week,month,...,years,hour_float,day_float,week_float,month_float,year_float,log2_accuracy,log10_accuracy,log2_accuracy_int,log10_accuracy_int
301,301,7.3815,3.2452,12,467320,4836326665,13,3,47,11,...,1,13.666667,3.527778,47.361111,11.817593,1.889117,35.849625,10.791812,35,10
2339,2339,7.3378,3.0791,8,215857,6676464189,22,3,22,5,...,1,22.616667,3.900694,22.414385,5.996690,1.410687,30.000000,9.030900,30,9
2984,2984,7.2308,3.0956,61,588233,8100169726,12,3,7,2,...,2,12.883333,3.495139,7.356448,2.616505,2.119165,59.307373,17.853298,59,17
2993,2993,7.1892,3.2495,13,47081,7896926684,17,5,5,2,...,1,17.683333,5.695139,5.670734,2.089838,1.089576,37.004397,11.139434,37,11
3336,3336,7.3740,3.0347,76,649934,6144260946,9,4,13,4,...,2,9.233333,4.343056,13.477579,4.044769,2.236556,62.479275,18.808136,62,18


In [63]:
def prepare_data(df, n_cell_x, n_cell_y):
    """
    Feature engineering and computation of the grid.
    """
    #Creating the grid
    size_x = 1.0 / n_cell_x
    size_y = 1.0 / n_cell_y
    eps = 0.00001  
    xs = np.where(df.x.values < eps, 0, df.x.values - eps)
    ys = np.where(df.y.values < eps, 0, df.y.values - eps)
    pos_x = (xs / size_x).astype(np.int)
    pos_y = (ys / size_y).astype(np.int)
    df['grid_cell'] = pos_y * n_cell_x + pos_x
    
    #Feature engineering
    fw = [500, 1000, 4, 3, 1./22., 2, 10, 8] #feature weights (black magic here)
    df.x = df.x.values * fw[0]
    df.y = df.y.values * fw[1]
    initial_date = np.datetime64('2014-01-01T01:01', dtype='datetime64[m]') 
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df.time.values)    
    df['hour'] = d_times.hour * fw[2]
    df['weekday'] = d_times.weekday * fw[3]
    df['day'] = (d_times.dayofyear * fw[4]).astype(int)
    df['month'] = d_times.month * fw[5]
    df['year'] = (d_times.year - 2013) * fw[6]
    df['log2_accuracy'] = np.log2(df.accuracy) * 10 * fw[7]
    df = df.drop(['time'], axis=1) 
    return df
    

def process_one_cell(df_train, df_test, grid_id, th):
    """   
    Classification inside one grid cell.
    """   
    #Working on df_train
    df_cell_train = df_train.loc[df_train.grid_cell == grid_id]
    place_counts = df_cell_train.place_id.value_counts()
    mask = (place_counts[df_cell_train.place_id.values] >= th).values
    df_cell_train = df_cell_train.loc[mask]

    #Working on df_test
    df_cell_test = df_test.loc[df_test.grid_cell == grid_id]
    row_ids = df_cell_test.index
    
    #Preparing data
    le = LabelEncoder()
    y = le.fit_transform(df_cell_train.place_id.values)
    X = df_cell_train.drop(['place_id', 'grid_cell'], axis=1).values.astype(int)
    X_test = df_cell_test.drop(['grid_cell'], axis = 1).values.astype(int)
    
    #Applying the classifier
    clf = KNeighborsClassifier(n_neighbors=25, weights='distance', 
                               metric='manhattan')
    clf.fit(X, y)
    y_pred = clf.predict_proba(X_test)
    pred_labels = le.inverse_transform(np.argsort(y_pred, axis=1)[:,::-1][:,:3])    
    return pred_labels, row_ids

def process_grid(df_train, df_test, th, n_cells):
    """
    Iterates over all grid cells, aggregates the results and makes the
    submission.
    """ 
    preds = {}
    
    for g_id in range(n_cells):
        if g_id % 100 == 0:
            print('iter: %s' %(g_id))
        
        #Applying classifier to one grid cell
        pred_labels, row_ids = process_one_cell(df_train, df_test, g_id, th)

        #Updating predictions
        for row_id, pred_label in zip(row_ids, pred_labels):
            preds[row_id] = pred_label

    return preds
    #print('Generating submission file ...')
    #Auxiliary dataframe with the 3 best predictions for each sample
    #df_aux = pd.DataFrame(preds, dtype=str, columns=['l1', 'l2', 'l3'])  
    
    #Concatenating the 3 predictions for each sample
    #ds_sub = df_aux.l1.str.cat([df_aux.l2, df_aux.l3], sep=' ')
    
    #Writting to csv
    #ds_sub.name = 'place_id'
    #ds_sub.to_csv('sub_knn.csv', index=True, header=True, index_label='row_id') 
      


In [76]:
n_cell_x = 2
n_cell_y = 4

df_train3 = df_train.loc[:, ['row_id','x','y','time','place_id', 'accuracy']]
df_train2, df_test2 = train_test_split(df_train3[(df_train3.x <= 1.0) & (df_train3.y <= 1.0)], random_state=88)

print('Preparing train data')
df_train2 = prepare_data(df_train2, n_cell_x, n_cell_y)

org = dict(zip(df_test2.index, df_test2.place_id))
print('Preparing test data')
df_test2 = prepare_data(df_test2, n_cell_x, n_cell_y)
df_test2 = df_test2.drop('place_id', 1)

#Solving classification problems inside each grid cell
th = 5 #Keeping place_ids with more than th samples.   
result = process_grid(df_train2, df_test2, th, n_cell_x*n_cell_y, )
cc = 0
assert set(org.keys()) == set(result.keys())
for row_id, place_id in result.iteritems():
    if org[row_id] == place_id[0]:
        cc += 1

score = cc / float(len(org))
print score

Preparing train data
Preparing test data
iter: 0
0.0165168812242
